# <font style="color:blue">Project 2: Kaggle Competition - Classification</font>

#### Maximum Points: 100

<div>
    <table>
        <tr><td><h3>Sr. no.</h3></td> <td><h3>Section</h3></td> <td><h3>Points</h3></td> </tr>
        <tr><td><h3>1</h3></td> <td><h3>Data Loader</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>2</h3></td> <td><h3>Configuration</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>3</h3></td> <td><h3>Evaluation Metric</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>4</h3></td> <td><h3>Train and Validation</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>5</h3></td> <td><h3>Model</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>6</h3></td> <td><h3>Utils</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>7</h3></td> <td><h3>Experiment</h3></td><td><h3>5</h3></td> </tr>
        <tr><td><h3>8</h3></td> <td><h3>TensorBoard Dev Scalars Log Link</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>9</h3></td> <td><h3>Kaggle Profile Link</h3></td> <td><h3>50</h3></td> </tr>
    </table>
</div>


## <font style="color:green">1. Data Loader [10 Points]</font>

In this section, you have to write a class or methods, which will be used to get training and validation data loader.

You need to write a custom dataset class to load data.

**Note; There is   no separate validation data. , You will thus have to create your own validation set, by dividing the train data into train and validation data. Usually, we do 80:20 ratio for train and validation, respectively.**


For example:

```python
class KenyanFood13Dataset(Dataset):
    """
    
    """
    
    def __init__(self, *args):
    ....
    ...
    
    def __getitem__(self, idx):
    ...
    ...
    
    
```

```
def get_data(args1, *agrs):
    ....
    ....
    return train_loader, test_loader
```

**class_counts: (very uneven)**
1. chapati        862
1. nyamachoma     784
1. bhaji          632
1. ugali          628
1. mandazi        620
1. kachumbari     494
1. matoke         483
1. githeri        479
1. masalachips    438
1. sukumawiki     402
1. pilau          329
1. mukimo         212
1. kukuchoma      173

Name: class, dtype: int64, sum: 6536



In [224]:
%matplotlib inline

In [225]:
# Import neccesaary libraries
import os
import time
import collections
from dataclasses import dataclass

# third party library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image

# Pytorch related
import torch
from torch.utils.data import Dataset
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from torch.optim import SGD  
from torch.utils.tensorboard import SummaryWriter
from torchvision.io import read_image
from torchvision import transforms
from torchvision.models import resnet152, resnet18

In [226]:
# all the Transforms
# def resize_preprocess():
#     """Compulsory transforms image to same_size and center cropped (not changing to Tensor yet)"""
#     resize_preprocess = transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#     ])
    
#     return resize_preprocess


def small_image_preprocess_transforms():
    """pre_process for KenyanFood13Testset holding original database(No need to hold big image)"""
    small_image_preprocess = transforms.Compose([
        transforms.Resize(1),
        transforms.ToTensor()
    ])
    
    return small_image_preprocess

### from here to below will change to Tensor
def image_preprocess_transforms():
    """pre_process for KenyanFood13Testset """
    image_preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
    
    return image_preprocess



def train_preprocess():
    """resize_preprocess() + couple transformation to improve accuracy in training, ToTensor and Normalization"""
    transforms_train = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(p=0.5),
        # Somehow this doesn't work
        #transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio = (0.33, 0.33), value= 0, inplace = False), 
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
        transforms.RandomRotation(degrees=10),
        transforms.RandomPosterize(bits=2),
        transforms.ToTensor(),
        transforms.Normalize(tc.mean, tc.std)        



    ])
    return transforms_train
    
def validation_preprocess():
    """ image_preprocess() + Normalization"""
    validation_train = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(tc.mean, tc.std)
    ])
    return validation_train



In [227]:
# Datasets Classes
class KenyanFood13Dataset(Dataset):
    """create KenyanFood Dataset from annotations_file and images"""
    def __init__(self, transform = None):
        self.img_labels = pd.read_csv(tc.annotation_file)
        self.img_dir = tc.img_dir
        # Make list of the classes list will give index (iterable) -> 13 classes
        self.classes = list(self.img_labels['class'].unique()) 
        # Make dictionary set of the classes 
        self.dict_classes = dict(enumerate(self.img_labels['class'].unique()))
        self.transform = transform
        
        # weigths of each class normalized (total = 1.0)
        def _inverse_ratio():
            """calculate inverse ratio of number of each classes for weighted sampler usage"""
            series = self.img_labels['class'].value_counts()
            total = series.sum()
            for index, _ in enumerate(series):
                series.iloc[index] = series.iloc[index]/total
            return series
        
        # This inner method won't work for inheritance since it is inside __init__()
        def _calc_sample_weights():
            class_counts = self.img_labels['class'].value_counts()
            sample_weights = [1/class_counts[i] for i in self.img_labels['class']]
            return sample_weights
        
        # sample_weights
        self.samples_weights = [self.img_labels['class'].value_counts()[i] for i in self.img_labels['class']]
        
        
    def _get_img_labels(self):
        return self.img_labels  
        
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, index):
        """Return (image, target) after resizing and preprocessing 
        iloc[index, 0] will return ids (e.g.,14278962987112149800) of each row"""
        #print(f"index: {index} \n len of self.img_labels from __getitem__:{len(self.img_labels)}")
        img_path = os.path.join(self.img_dir, str(self.img_labels.iloc[index, 0])+".jpg")
#         print(f"Path created:{img_path}")        
        image = Image.open(img_path)
#         print(f"image found:{image}")  
        # label is string so will return index (pytorch cannot make string to tensor)
        # iloc[index, 1] will return the class_name (e.g., githeri) for id in [index, 1] (e.g., 14278962987112149800)
        label_index = self.class2index(self.img_labels.iloc[index, 1])  # returns int

        if self.transform:
            image = self.transform(image)

        return image, label_index #image: Image or Tensor ,  label_index: int
    
    
    def class2index(self, class_name:str)-> int:
        """Returns the index of a given class."""
        return self.classes.index(class_name)
    
    
    def index2class(self, class_index:int)-> str:
        """Returns the class of a given index."""
        return self.classes[class_index]


class KenyanFood13SplitDataset(KenyanFood13Dataset):
    def __init__(self, transform = None, train= True):
        super().__init__(transform = transform)
        #self.img_labels = None
#         print(f"self.img_labels in __init()__ first: {self.img_labels}\n") # 6536
        image_labels = super()._get_img_labels()

        if train:
            self.img_labels = image_labels.iloc[:int(len(image_labels) * tc.train_split)]
#             print(f"len of self.img_labels in __init__ () of subclass should be 5228 :{len(self.img_labels)}")# 5228
        else:
            self.img_labels = image_labels.iloc[int(len(image_labels) * tc.train_split):]

#         print(f"self.img_labels in __init()__ last subclass should be 5228:  {len(self.img_labels)}")#  5228
        
        # Override the sample_weights since train database has splitted database, needed for weighted Random Sampler
        self.samples_weights = [self.img_labels['class'].value_counts()[i] for i in self.img_labels['class']]
    
class KenyanFood13Testset(Dataset):
    """Kenyan food test dataset, contains original KenanFood13Dataset for image2class and class2image methods"""
    def __init__(self, transform = image_preprocess_transforms()):
        self.img_dir = tc.img_dir
        self.test_labels = pd.read_csv(tc.test_csv_file)
        self.transform = transform
        # this is to use class2index() and class2index()
        self.base_dataset = KenyanFood13Dataset(small_image_preprocess_transforms())

    def __getitem__(self, index):
        """Retrieves one item from the dataset."""
        
        # get img path from test_labels. img_dir + id in csv file (per index) + ".jpg" 
        img = os.path.join(self.img_dir, str(self.test_labels.iloc[index, 0]) + '.jpg')
        
        image = Image.open(img)

        if self.transform:
            image = self.transform(image)

        return image
    
    def __len__(self):
        return len(self.test_labels)

## <font style="color:green">2. Configuration [5 Points]</font>

**Define your configuration here.**

For example:


```python
@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 10 
    epochs_count: int = 50  
    init_learning_rate: float = 0.1  # initial learning rate for lr scheduler
    log_interval: int = 5  
    test_interval: int = 1  
    data_root: str = "/kaggle/input/pytorch-opencv-course-classification/" 
    num_workers: int = 2  
    device: str = 'cuda'  
    
```

In [237]:
# Settings and configurations
@dataclass
class SystemConfiguration:
    seed:int = 21
        
# Specifiy all the data needed in dataclass named TrainingConfiguration
@dataclass
class TrainingConfiguration:
    batch_size: int = 52
    epoch_count: int = 20
    init_learning_rate: float = 0.001
    log_interval: int = 5
    test_interval: int = 1
    data_root: str = "/kaggle/input/opencv-pytorch-dl-course-classification/"
    model_dir: str = '/kaggle/working/models/'
    log_dir: str = '/kaggle/working/logs'
    annotation_file: str = data_root + "train.csv"
    test_csv_file: str = data_root + "test.csv"
    img_dir: str = data_root + "images/images/"
    submission_csv: str = '/kaggle/working/submission.csv'
    num_workers: int = 2
    device: str = 'cuda'
    train_split: float = 0.8
    random_seed: int = 42
    model_name: str = "resnet152"
    lr: float =0.00001
    classes: tuple = ('githeri', 'ugali', 'kachumbari', 'matoke', 'sukumawiki', 'bhaji', 'mandazi', 'kukuchoma', 'nyamachoma', 'pilau', 'chapati', 'masalachips', 'mukimo')
    dataset_len: int = 6536
    #below is calculated from KenyanFood13 pictures
    #mean: torch.tensor = torch.tensor([0.5768, 0.4622, 0.3460])
    #std: torch.tensor = torch.tensor([0.2699, 0.2739, 0.2826])
    
    #below mean and standard are for Resnet18 https://pytorch.org/hub/pytorch_vision_resnet/
    mean: torch.tensor = torch.tensor([0.485, 0.456, 0.406])
    std: torch.tensor  = torch.tensor([0.229, 0.224, 0.225])
        
    tb_writer: SummaryWriter = SummaryWriter(log_dir)
    criterion: torch.nn.CrossEntropyLoss = torch.nn.CrossEntropyLoss()        

def setup_system(SystemConfiguration):
    torch.manual_seed(system_config.seed)
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        torch.backends.cudnn_benchmark_enabled = True
        torch.backends.cudnn.deterministic = True
        
        
tc = TrainingConfiguration()
system_config = SystemConfiguration()
setup_system(system_config)


for path in [tc.log_dir, tc.model_dir]:
    if not os.path.exists(path):
        os.makedirs(path)


## <font style="color:green">3. Evaluation Metric [10 Points]</font>

**Define methods or classes that will be used in model evaluation. For example, accuracy, f1-score etc.**

## <font style="color:green">4. Train and Validation [5 Points]</font>


**Write the methods or classes to be used for training and validation.**

In [229]:
def train(model, train_loader, optimizer) -> float:
    """train by model by given train_loader(and its datasets) and optimizer, return average loss and accuracy"""
    total_loss = 0.0
    total_acc = 0
    count = 0
    model.train()
    
    for inputs, labels in tqdm(train_loader):
        count += len(labels)
        # move to target device (GPU or CPU)
        inputs = inputs.to(tc.device)
        labels = labels.to(tc.device)
        
        # prediction
        outputs = model(inputs)
        # calculate loss
        loss = tc.criterion(outputs, labels)
        # initialize the gradients -> calucalte the gradients -> Update the gradients
        optimizer.zero_grad()     
        loss.backward()
        optimizer.step()
        
        # adds up loss pf number of batches
        total_loss += loss.item()
        # calculates accumulative average loss by dividing total loss by length of dataset predicted so far
        avg_loss = total_loss / count
        
        # get highest predicted value
        predicted = torch.max(outputs, axis=1)[1]
        # calculates accumulative average accuracy by dividing total loss by length of dataset predicted so far
        total_acc  += (predicted == labels).sum().item()
        avg_acc =  total_acc / count
        
    return avg_loss, avg_acc

In [230]:
def validate(model, validation_loader):
    """validate by model by given validation_loader(and its datasets) , return average loss"""
    total_loss = 0.0
    total_acc = 0
    count = 0
    model.eval()
    
    with torch.no_grad():
        for inputs, labels in tqdm(validation_loader):
            count += len(labels)
            # move to target device (GPU or CPU)
            inputs = inputs.to(tc.device)
            labels = labels.to(tc.device)
            
            # prediction
            outputs = model(inputs)
            # calculate loss
            loss = tc.criterion(outputs, labels)
            
            # adds up loss pf number of batches
            total_loss += loss.item()
            # calculates accumulative average loss by dividing total loss by length of dataset predicted so far
            avg_loss = total_loss / count
            
            # get highest predicted value
            predicted = torch.max(outputs, axis=1)[1]
            # calculate number of correct answer
            total_acc  += (predicted == labels).sum().item()
            # calculates accumulative average accuracy by dividing total loss by length of dataset predicted so far
            avg_acc =  total_acc / count
        
            
    return avg_loss, avg_acc
        

In [231]:
def predict(trained_model, prediction_loader) -> list:
    """prediction with model and test_loader, returns list of prediction result, assume to be used for inference ONLY"""  
    predictions = torch.tensor([]).to(tc.device)
    for inputs in prediction_loader:
        inputs = inputs.to(tc.device)
        outputs = trained_model(inputs)
        # get the predicted result by index number, each prediction will be done by tc.batch_size so predicted is array of batch_size
        predictions = torch.hstack((predictions, torch.max(outputs, axis=1)[1]))
        
    return predictions.tolist()


## <font style="color:green">5. Model [5 Points]</font>

**Define your model in this section.**

**You are allowed to use any pre-trained model.**

In [246]:
# # Resnet 18 (or 152) and change last layer output to 13
# model = resnet18(pretrained = True)
# # Change the last output to 13
# fc_in_features = model.fc.in_features
# model.fc = torch.nn.Linear(fc_in_features, (len(tc.classes)))
# model = model.to(tc.device)

In [243]:
# Efficientnet and change last layer output to 13
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b7')
fc_in_features = model._fc.in_features
model.fc = torch.nn.Linear(fc_in_features, (len(tc.classes)))
model = model.to(tc.device)

Loaded pretrained weights for efficientnet-b7


## <font style="color:green">6. Utils [5 Points]</font>

**Define those methods or classes, which have  not been covered in the above sections.**

In [233]:
## util functions

def get_mean_std(dataset):
    """returns mean and standard deviation of dataset given, since this method will take long time, once calcuated, not been used"""
    # calculated mean: tensor([0.5768, 0.4622, 0.3460]), std:tensor([0.2699, 0.2739, 0.2826]):
    loader = DataLoader(dataset)
    
    batch_mean = torch.zeros(3) # tensor([0,0,0])
    batch_mean_sqrd = torch.zeros(3)
    
    for batch_data, _ in loader:
        batch_mean += batch_data.mean(dim=(0,2,3))
        batch_mean_sqrd += (batch_data **2).mean(dim=(0,2,3))
        
    mean = batch_mean / len(loader)
    var = (batch_mean_sqrd) / len(loader) - (mean **2)
    
    std = var ** .5
    
    print("mean: {}, std:{}:".format(mean, std))
    return mean, std


def eval_loss(data_loader, device, model, criterion):
    """evaluate losses and can use this for visualize"""
    for images, labales in data_loader:
        break
        
    inputs = images.to(device)
    labels = labels.to(device)
    
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    return loss

def evaluate_history(history):
    import matplotlib.pyplot as plt
    print("initial stats: Loss{:.5f}  Accuracy{:.5f}".format(history[0,3], history[0,4]))
    print("final stats: Loss{:.5f}  Accuracy{:.5f}".format(history[-1,3], history[-1,4]))
    
    num_epochs = len(history)
    unit = num_epochs/10
    
    # Display learning curve (Loss)
    plt.figure(figsize=(9,8))
    plt.plot(history[:,0], history[:,1], 'b', label='train')
    plt.plot(history[:,0], history[:,3], 'k', label='validation')
    plt.xticks(np.arange(0,num_epochs+1, unit))
    plt.xlabel('# of iteration')
    plt.ylabel('loss')
    plt.title('learning curve (Loss)')
    plt.legend()
    plt.show()

    # Display learning curve (Accuracy)
    plt.figure(figsize=(9,8))
    plt.plot(history[:,0], history[:,2], 'b', label='train')
    plt.plot(history[:,0], history[:,4], 'k', label='validation')
    plt.xticks(np.arange(0,num_epochs+1,unit))
    plt.xlabel('# of iteration')
    plt.ylabel('loss')
    plt.title('learning curve (Accuracy')
    plt.legend()
    plt.show()


# model save and load functions
def save_model(model, device):
    if not os.path.exists(tc.model_dir):
        os.makedirs(tc.model_dir)
    
    model_path = os.path.join(tc.model_dir, tc.model_name)
    
    if device == "cuda":
        model.to("cpu")
    
    torch.save(model.state_dict(), model_path + "best_model.pt")
    
    if device == "cuda":
        model.to("cuda")
    
    return

def load_model(model):
    
    model_path = os.path.join(tc.model_dir, tc.model_name)
    model.load_state_dict(torch.load(model_path + 'best_model.pt'))
    
    return model

* id: 0 'githeri': 479,
* id: 1 'ugali': 628,
* id: 2 'kachumbari': 494,
* id: 3 'matoke': 483,
* id: 4 'sukumawiki': 402,
* id: 5 'bhaji': 632,
* id: 6 'mandazi': 620,
* id: 7 'kukuchoma': 173,
* id: 8 'nyamachoma': 784,
* id: 9 'pilau': 329,
* id:10 'chapati': 862,
* id:11 'masalachips': 438,
* id:13 'mukimo': 212

In [234]:
from torch.utils.data.sampler import BatchSampler

class BalancedBatchSampler(BatchSampler):
    
    
    def __init__(self, dataset, n_classes, n_samples):
        loader = DataLoader(dataset)
        # keep ALL the label as list
        self.labels_list = []
        for _, label in loader:
            self.labels_list.append(label)
        # and keep it as Tensor
        self.labels = torch.LongTensor(self.labels_list)
        
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.dataset = dataset
        self.batch_size = self.n_samples * self.n_classes
        
    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < len(self.dataset):
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return len(self.dataset) // self.batch_size

In [235]:
def visualize_batch_data(dataloader, with_visual_chart = True , with_class_string = True) -> dict:
    """visualize what class is in the batch to visually see the equalness of the samples"""
    #{0: 'githeri', 1: 'ugali', 2: 'kachumbari', 3: 'matoke', 4: 'sukumawiki', 5: 'bhaji', 6: 'mandazi', 7: 'kukuchoma', 8: 'nyamachoma', 9: 'pilau', 10: 'chapati', 11: 'masalachips', 12: 'mukimo'}
    # total_number_appeared: Counter({10: 862, 8: 784, 5: 632, 1: 628, 6: 620, 2: 494, 3: 483, 0: 479, 11: 438, 4: 402, 9: 329, 12: 212, 7: 173})
    total_num_images = len(data_loader.dataset)
    total_number_appeared = collections.Counter()
    number_appeared_per_batch = []
    dict_classes = data_loader.dataset.dict_classes
    
    for index, (_, label_index) in enumerate(data_loader): # don't use image
        class_ids, class_counts = torch.unique(label_index, return_counts=True) # return in torch.tensor
        number_appeared = collections.Counter(label_index.tolist())
        print(f"batch:{index} number_appeared:{number_appeared}")
        
        # add zero for sample which was not picked, and add that label_index:0 to Counter
        while len(number_appeared) < len(dict_classes):
            for i in range(len(number_appeared)):
                number_appeared.setdefault(i,0)
            print(f"batch:{index} number_appeared_after_setdefault:{number_appeared}")
        # for total count
        total_number_appeared += number_appeared

            
        # sort based on keys (label_index) and cast to dict for visualization
        dict_number_appeared = dict(sorted(number_appeared.items()))
        #print(f"dict_number_appeared:{dict_number_appeared}")
        number_appeared_per_batch.append(dict_number_appeared)
        
    if with_visual_chart:
        """create bar graph for all batches which class had samples"""
        fig, axes = plt.subplots(int(tc.batch_size/4), 4 , figsize = (20, 20))
        for i in range(int(tc.batch_size/4)):
            for j in range(4):
                axes[i][j].bar(number_appeared_per_batch[int(j+(i*4))].keys(), list(number_appeared_per_batch[int(j+(i*4))].values()))
                if with_class_string:
                    axes[i][j].set_xticks(list(range(len(dict_classes))))
                    axes[i][j].set_xticklabels(list(dict_classes.values()))
                else:
                    axes[i][j].set_xticks(list(range(len(dict_classes))))
                    axes[i][j].set_xticklabels(list(dict_classes.keys()))

    
    # return dict type, key=class name, value=how many times appeared
    if with_class_string:
        return {data_loader.dataset.index2class(key):value for key, value in total_number_appeared.items()}
    
    # return dict type, key=class_id, value=how many times appeared
    return dict(total_number_appeared)


## <font style="color:green">7. Experiment [5 Points]</font>

**Choose your optimizer and LR-scheduler and use the above methods and classes to train your model.**

In [247]:
# split dataset into train & validation dataset based on split ratio
kenyan_dataset = KenyanFood13Dataset() 
train_dataset = KenyanFood13SplitDataset(transform = train_preprocess(),train = True)
validation_dataset = KenyanFood13SplitDataset(transform = validation_preprocess(), train = False)
sequential_sampler = torch.utils.data.SequentialSampler(train_dataset)
weight_sampler = WeightedRandomSampler(weights=train_dataset.samples_weights, num_samples=len(train_dataset), replacement=True)
train_loader = DataLoader(train_dataset, sampler = weight_sampler, batch_size = tc.batch_size, shuffle = False)
validation_loader = DataLoader(validation_dataset, batch_size = tc.batch_size, shuffle = False )

print(f"length of train_dataset {len(train_dataset.samples_weights)}")

length of train_dataset 5228


In [248]:
best_val_loss = np.Inf
history = np.zeros((0,6))
for epoch in range(tc.epoch_count):
    # time one cycle of train and validate
    start_time = time.time()
    train_loss, train_acc = train(model, train_loader, optimizer)
    val_loss, val_acc = validate(model, validation_loader)
    elapsed_time = time.time() - start_time
    
    # save the weight when the loss gets smaller than last one
    if val_loss < best_val_loss:
        save_model(model, tc.device)
        print("WEIGHTS-ARE-SAVED")
        best_val_loss = val_loss
        
    print(f"Epoch{epoch+1}/{tc.epoch_count}, loss:{train_loss:.5f}, acc:{train_acc:.5f}, val_loss:{val_loss:.5f}, val_acc:{val_acc:.5f}, elapsed_time:{elapsed_time:.5f}")
    # store 0:epoch, 1:avg_train_loss, 2: avg_train_acc, 3: avg_val_loss, 4: avg_val_acc, 5: elapsed_time
    item = np.array([epoch+1, train_loss, train_acc, val_loss, val_acc, elapsed_time])
    # store history for matplotlib visualization
    history = np.vstack((history, item))

    # For tensorboard
    tc.tb_writer.add_scalar('Loss/Train within {}/{}'.format(epoch+1,tc.epoch_count), train_loss, epoch+1)
    tc.tb_writer.add_scalar('Accuracy/Train within {}/{}'.format(epoch+1,tc.epoch_count), train_acc, epoch+1)
    tc.tb_writer.add_scalar('Time/elapsed_time', elapsed_time, epoch+1)

  0%|          | 0/101 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 160.00 MiB (GPU 0; 15.90 GiB total capacity; 14.45 GiB already allocated; 81.75 MiB free; 14.89 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Now do the inference and get submission.csv file
test_dataset = KenyanFood13Testset()
test_loader = DataLoader(test_dataset, batch_size = tc.batch_size, shuffle = False)

# prediction
predicted_result = predict(model, test_loader).tolist()
#print("predicted_result len:{}".format(len(predicted_result)))
classifications = []

#change predicted index to class_names
for index in range(len(predicted_result)):
    predicted_result_class = test_dataset.base_dataset.index2class(int(predicted_result[index]))
    classifications.append(predicted_result_class)

# put it to csv file
classes = pd.DataFrame(classifications, columns = ["class"])
result = test_dataset.test_labels.join(classes)
result.to_csv(tc.submission_csv, index=False, header = True)

In [ ]:
def fit(model, optimizer, train_loader, validation_loader, history):
    """works like tensorflow fits. epoch_count, optimizer and criterion(e.g., CrossEntorpyLoss) from training config"""
    
    #from tqdm.notebook import tqdm
    
    base_epochs = len(history)
    
    for epoch in range(base_epochs, tc.epoch_count + base_epochs):
        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0
        best_valid_loss = np.Inf
        
        # training Phase
        model.train()
        count = 0
        
        t_begin = time.time()
        for inputs, labels in train_loader:
            
            count += len(labels)
            inputs = inputs.to(tc.device)
            labels = labels.to(tc.device)
            
            # initialize the gradients
            optimizer.zero_grad()
            # predict
            outputs = model(inputs)
            # calculate loss
            loss = tc.criterion(outputs, labels)
            train_loss += loss.item()
            
            #calucalte the graidents
            loss.backward()
            #update parameter
            optimizer.step()
            
            # get highest predicted value
            predicted = torch.max(outputs, axis=1)[1]
            # calculate number of correct answer
            train_acc  += (predicted == labels).sum().item()
            
            #calculate the average loss and accuracy in the batch
            avg_train_loss = train_loss / count
            avg_train_acc = train_acc / count
            
        #evaluation phase
        model.eval()
        count = 0
        
        for inputs, labels in validation_loader:
            count += len(labels)
            inputs = inputs.to(tc.device)
            labels = labels.to(tc.device)
            
            # calculate the prediction
            outputs = model(inputs)
            
            # calculate the loss
            loss = tc.criterion(outputs, labels)
            val_loss += loss.item()
            
            if val_loss < best_valid_loss:
                torch.save(model.state_dict(), "best_model.pt")
                print("WEIGHTS-ARE-SAVED")
                best_valid_loss = val_loss
                
            # get highest predicted value
            predicted = torch.max(outputs, axis=1)[1]
            # calculate number of correct answer
            val_acc += (predicted == labels).sum().item()
            
            #calculate the average loss and accuracy in the batch
            avg_val_loss = val_loss / count
            avg_val_acc = val_acc / count
            
            
        elapsed_time = time.time() - t_begin
        # print out
        print("Epoch{}/{},  loss:{:.5f}, acc:{:.5f}, val_loss:{:.5f}, val_acc:{:.5f}, elapsed_time:{:.5f}".format(
            epoch+1,tc.epoch_count+base_epochs, avg_train_loss, avg_train_acc, avg_val_loss, avg_val_acc, elapsed_time
        ))
        # store 0:epoch, 1:avg_train_loss, 2: avg_train_acc, 3: avg_val_loss, 4: avg_val_acc, 5: elapsed_time
        item = np.array([epoch+1, avg_train_loss, avg_train_acc, avg_val_loss, avg_val_acc, elapsed_time])
        history = np.vstack((history, item))
        
        # For tensorboard
        tc.tb_writer.add_scalar('Loss/Train within {}/{}'.format(epoch+1,tc.epoch_count+base_epochs), avg_train_loss, epoch+1)
        tc.tb_writer.add_scalar('Accuracy/Train within {}/{}'.format(epoch+1,tc.epoch_count+base_epochs), avg_train_acc, epoch+1)
        tc.tb_writer.add_scalar('Time/elapsed_time', elapsed_time, epoch+1)

        
    return history

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = tc.batch_size, shuffle = True)
validation_loader = DataLoader(validation_dataset, batch_size = tc.batch_size, shuffle = False)
history = np.zeros((0,6)) # history to record 0: num of iteration, 1:training_loss, 2:trainin_acc, 3:val_loss, 4:val_acc 5:elapsed time
optimizer = SGD(model.parameters(), lr=tc.lr, momentum = 0.9)
history = fit(model, optimizer, train_loader, validation_loader, history)

## <font style="color:green">8. TensorBoard Dev Scalars Log Link [5 Points]</font>

**Share your TensorBoard scalars logs link here You can also share (not mandatory) your GitHub link, if you have pushed this project in GitHub.**


For example, [Find Project2 logs here](https://tensorboard.dev/experiment/kMJ4YU0wSNG0IkjrluQ5Dg/#scalars).

## <font style="color:green">9. Kaggle Profile Link [50 Points]</font>

**Share your Kaggle profile link  with us here to score , points in  the competition.**

**For full points, you need a minimum accuracy of `75%` on the test data. If accuracy is less than `70%`, you gain  no points for this section.**


**Submit `submission.csv` (prediction for images in `test.csv`), in the `Submit Predictions` tab in Kaggle, to get evaluated for  this section.**